In [1]:
from tsp import TSP
from tsp.solvers import NNHead, RandomSolver

In [34]:
instance_a = TSP.from_csv('../data/TSPA.csv')

In [ ]:
solver = RandomSolver(instance_a)
solution = solver.solve()
instance_a.visualize(solution)

In [ ]:
solver = NNHead(instance_a, starting_node=0)
solution = solver.solve()
instance_a.visualize(solution)